## 5.8 网络中的网络（NiN）

### 5.8.1 NiN 快

In [1]:
import tensorflow as tf

/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [2]:
def nin_block(num_channels, kernel_size, strides, padding):
    def layer(xn):
        xn = tf.keras.layers.Conv2D(
            num_channels, kernel_size, strides=strides, padding=padding, activation='relu')(xn)
        xn = tf.keras.layers.Conv2D(num_channels, (1, 1), activation='relu')(xn)
        xn = tf.keras.layers.Conv2D(num_channels, (1, 1), activation='relu')(xn)
        return xn
    return layer
        

### 5.8.2 NiN模型

In [3]:
inputs = tf.keras.layers.Input((224, 224, 1))
x = nin_block(96, (11, 11), strides=4, padding='valid')(inputs)
x = tf.keras.layers.MaxPool2D(3, strides=2)(x)
x = nin_block(256, (5, 5), strides=1, padding='same')(x)
x = tf.keras.layers.MaxPool2D(3, strides=2)(x)
x = nin_block(384, (3, 3), strides=1, padding='same')(x)
x = tf.keras.layers.MaxPool2D(3, strides=2)(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = nin_block(10, (3, 3), strides=1, padding='same')(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Flatten()(x)

model = tf.keras.Model(inputs, x)
model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 1)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 54, 54, 96)        11712     
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 54, 54, 96)        9312      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 54, 54, 96)        9312      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 26, 26, 96)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 26, 26, 256)       614656    
_________

### 5.8.3 获取数据和训练模型
(略)